In [1]:
from tqdm import tqdm
import pymssql
import pandas as pd
import numpy as np
import warnings
import glob 
import os

warnings.filterwarnings('ignore')

In [2]:
conn = pymssql.connect(server='AX-SQL', database='Staging')
cursor = conn.cursor()

sql = f'''
SELECT [Код склада], [Код товара], [Артикул поставщика], [Внутренний артикул], [ID Сайта], [Товарное направление],
[Товарная группа], [Товарный кластер], [Проба], [Размер изделия], [Тип изделия 1], [Тип изделия 2],
[Тип изделия 3], [Дизайн], [Бренд], [Гендерный признак], [Ценовая корзина], [Группа цен], [Остаток],
[Чистый вес], [Цена изделия на бирке], [Скидка на изделии], [Запрет скидки], [Цена Закупки за 1 шт. (упр сс с НДС за единицу)], [Дата закупки]
from [Staging].[Reports].[Remainings_8h]
where [Товарное направление] in (N'БК', N'ЦБ', N'ИФ', N'ПДК', N'ДК')
'''

sql_query = pd.read_sql_query(sql, conn)
stock = pd.DataFrame(sql_query)

In [3]:
images = pd.read_excel(r'C:\Users\Trenkin.Sergey\Desktop\Артикул-фото 2024.06.11_.xlsx')

In [4]:
zapret = pd.read_excel(r'C:\Users\Trenkin.Sergey\Desktop\запрет Артикулы к размещению.xlsx')

In [5]:
kniga = pd.read_excel(r'C:\Users\Trenkin.Sergey\Desktop\Книга11.xlsx')

,Код склада,Дата открытия объекта,Код объекта,Город,Наименование объекта
0,4383,2018-05-14,3-04-039,Киселевск,"Киселевск, Ленина, 41"
1,496,2008-06-16,3-04-010,Междуреченск,"Междуреченск, Коммунистический, 26"
2,4783,2019-12-05,3-04-046,Новокузнецк,"Новокузнецк, Доз, 10а, ТРЦ Планета"
3,474,2008-03-17,3-04-009,Новокузнецк,"Новокузнецк, Ленина, 40"
4,479,2008-04-17,3-04-008,Новокузнецк,"Новокузнецк, Мориса Тореза, 75"
...,...,...,...,...,...
923,4918,2021-03-07,8-01-041,Санкт-Петербург,"Санкт-Петербург, Комендантский, 12"
924,4333,2018-12-11,8-01-021,Санкт-Петербург,"Санкт-Петербург, ТРЦ МЕГА Дыбенко"
925,5409,2021-10-03,8-01-049,Сланцы,"Сланцы, Ленина, 11"
926,5413,2021-09-30,8-01-051,Тихвин,"Тихвин, Карла Маркса, 50Б"


In [6]:
os.chdir(r'C:\\Users\Trenkin.Sergey\Desktop\sales')
extension = 'xlsx'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
all_filenames

['2024-03-10_2024-03-19.xlsx',
 '2024-03-20_2024-03-31.xlsx',
 '2024-04-01_2024-04-09.xlsx',
 '2024-04-10_2024-04-19.xlsx',
 '2024-04-20_2024-04-30.xlsx',
 '2024-05-01_2024-05-09.xlsx',
 '2024-05-10_2024-05-19.xlsx',
 '2024-06-01_2024-06-09.xlsx',
 '2024-06-10_2024-06-10.xlsx',
 '2024-06-10_2024-06-12.xlsx',
 'Книга13.xlsx']

In [7]:
sale = pd.DataFrame()
for name in tqdm(all_filenames):
    sales = pd.read_excel(name)
    sale = pd.concat([sales, sale], ignore_index= True)

100%|██████████| 11/11 [10:11<00:00, 55.60s/it]


In [9]:
sale['Дата'] = pd.to_datetime(sale['Дата'], dayfirst=True, format='mixed')

In [10]:
sale = sale[(sale['Дата'] >= pd.to_datetime('2024-03-14', dayfirst=True, format='mixed')) & 
            (sale['Дата'] <= pd.to_datetime('2024-06-11', dayfirst=True, format='mixed'))]

In [11]:
sale = sale.groupby(by = 'Тов', as_index=False)[['Количество', 'Общая сумма']].sum()

In [15]:
kniga['Код склада'] = kniga['Код склада'].astype(str)
skladi = kniga['Код склада'].unique().tolist()

In [12]:
stock.columns = ['code', 'Код товара', 'Артикул поставщика', 'Внутренний артикул',
    'ID Сайта', 'Товарное направление', 'Товарная группа',
    'Товарный кластер', 'Проба', 'Размер изделия', 'Тип изделия 1',
    'Тип изделия 2', 'Тип изделия 3', 'Дизайн', 'Бренд',
    'Гендерный признак', 'Ценовая корзина', 'Группа цен', 'Остаток',
    'Чистый вес', 'Цена изделия на бирке', 'Скидка на изделии',
    'Запрет скидки', 'Цена Закупки за 1 шт. (упр сс с НДС за единицу)',
    'Дата закупки']

In [16]:
stock = stock.query('code in @skladi')

In [17]:
stock.columns = ['Код склада', 'Код товара', 'Артикул поставщика', 'Внутренний артикул',
    'ID Сайта', 'Товарное направление', 'Товарная группа',
    'Товарный кластер', 'Проба', 'Размер изделия', 'Тип изделия 1',
    'Тип изделия 2', 'Тип изделия 3', 'Дизайн', 'Бренд',
    'Гендерный признак', 'Ценовая корзина', 'Группа цен', 'Остаток',
    'Чистый вес', 'Цена изделия на бирке', 'Скидка на изделии',
    'Запрет скидки', 'Цена Закупки за 1 шт. (упр сс с НДС за единицу)',
    'Дата закупки']

In [18]:
result = stock.drop_duplicates(subset=['Код товара'])

In [19]:
tovs = result['ID Сайта'].dropna().tolist()

In [20]:
images['article'] = images['article'].astype(str)

In [21]:
images = images.query('article in @tovs')

In [22]:
images = images[['article', 'published', 'Ссылка на изображение']]

In [23]:
images.columns = ['ID Сайта', 'Опубликован', 'Ссылка на изображение']

In [24]:
result = result.merge(images, how='outer', on='ID Сайта')

In [25]:
result = result.drop(columns = ['Остаток'])

In [26]:
df = stock

In [27]:
stock = df.groupby(by = 'Код товара', as_index=False)['Остаток'].sum()

In [28]:
result = result.merge(stock, how='left', on='Код товара')

In [29]:
sale.columns = ['Код товара', 'Количество продаж 14.03-11.06', 'Сумма продаж 14.03-11.06']

In [30]:
result = result.merge(sale, how='left', on='Код товара')

In [31]:
result.columns

Index(['Код склада', 'Код товара', 'Артикул поставщика', 'Внутренний артикул',
       'ID Сайта', 'Товарное направление', 'Товарная группа',
       'Товарный кластер', 'Проба', 'Размер изделия', 'Тип изделия 1',
       'Тип изделия 2', 'Тип изделия 3', 'Дизайн', 'Бренд',
       'Гендерный признак', 'Ценовая корзина', 'Группа цен', 'Чистый вес',
       'Цена изделия на бирке', 'Скидка на изделии', 'Запрет скидки',
       'Цена Закупки за 1 шт. (упр сс с НДС за единицу)', 'Дата закупки',
       'Опубликован', 'Ссылка на изображение', 'Остаток',
       'Количество продаж 14.03-11.06', 'Сумма продаж 14.03-11.06'],
      dtype='object')

In [32]:
result = result.drop_duplicates(subset=['Код товара'])

In [33]:
df['Чистый вес'] = df['Чистый вес'].astype(float)

In [34]:
clear_weight = df.groupby(by = 'Код товара', as_index=False)['Чистый вес'].agg({'min', 'median', 'max'})
clear_weight.columns = ['Код товара', 'Чистый вес мин', 'Чистый вес медиана', 'Чистый вес макс']
clear_weight['Чистый вес размах, %'] = (clear_weight['Чистый вес макс'] - clear_weight['Чистый вес мин']) / ((clear_weight['Чистый вес макс'] + clear_weight['Чистый вес мин']) / 2)

In [35]:
result = result.merge(clear_weight, how='left', on='Код товара')

In [36]:
price_birka = df.groupby(by = 'Код товара', as_index=False)['Цена изделия на бирке'].agg({'min', 'median', 'max'})
price_birka.columns = ['Код товара', 'Цена изделия на бирке мин', 'Цена изделия на бирке медиана', 'Цена изделия на бирке макс']

In [37]:
result = result.merge(price_birka, how='left', on='Код товара')

In [38]:
base_discount = df.groupby(by = 'Код товара', as_index=False)['Скидка на изделии'].agg({'min', 'max'})
base_discount.columns = ['Код товара', 'Базовая скидка мин', 'Базовая скидка макс']

In [39]:
result = result.merge(base_discount, how='left', on='Код товара')

In [40]:
df['Цена со скидками'] = np.where(
    df['Запрет скидки'] == 'Да', df['Цена изделия на бирке'], df['Цена изделия на бирке'] * 0.8 * 0.6 * (1 - df['Скидка на изделии'] / 100)
)

In [41]:
price_skidki = df.groupby(by = 'Код товара', as_index=False)['Цена со скидками'].agg({'min', 'median', 'max'})
price_skidki.columns = ['Код товара', 'Цена со скидками мин', 'Цена со скидками медиана', 'Цена со скидками макс']
price_skidki['Цена со скидками размах, %'] = (price_skidki['Цена со скидками макс'] - price_skidki['Цена со скидками мин']) / ((price_skidki['Цена со скидками макс'] + price_skidki['Цена со скидками мин']) / 2)

In [42]:
result = result.merge(price_skidki, how='left', on='Код товара')

In [43]:
selfprice = df.groupby(by = 'Код товара', as_index=False)['Цена Закупки за 1 шт. (упр сс с НДС за единицу)'].agg({'min', 'median', 'max'})
selfprice.columns = ['Код товара', 'Себестоимость мин', 'Себестоимость медиана', 'Себестоимость макс']
selfprice['Себестоимость размах, %'] = (selfprice['Себестоимость макс'] - selfprice['Себестоимость мин']) / ((selfprice['Себестоимость макс'] + selfprice['Себестоимость мин']) / 2)

In [44]:
result = result.merge(selfprice, how='left', on='Код товара')

In [45]:
df['Наценка'] = df['Цена со скидками'] / df['Цена Закупки за 1 шт. (упр сс с НДС за единицу)'] - 1

In [46]:
markup = df.groupby(by = 'Код товара', as_index=False)['Наценка'].agg({'min', 'median', 'max'})
markup.columns = ['Код товара', 'Наценка мин', 'Наценка медиана', 'Наценка макс']

In [47]:
result = result.merge(markup, how='left', on='Код товара')

In [48]:
zapret['Артикул сайта'] = zapret['Артикул сайта'].astype(str)

In [49]:
zapret = zapret['Артикул сайта'].tolist()

In [50]:
result['Запрет к размещению'] = np.nan
for i in result.index:
    if result['ID Сайта'][i] in zapret:
        result['Запрет к размещению'][i] = True
    else:
        result['Запрет к размещению'][i] = False

In [51]:
ecom = pd.read_excel(r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\МЮР\ecom_stock_raw\Остатки 2024.06.17.xlsx')

In [52]:
path = r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\discounts_zolotoy\\'
list_of_files = glob.glob(path + '\*.xlsx') 
latest_file = max(list_of_files, key=os.path.getctime)

discount_ecom = pd.read_excel(latest_file)

In [53]:
discount_ecom.columns = ['article', 'Опубликован', 'discount_value_promotions_1']

In [54]:
ecom = ecom.merge(discount_ecom, how='left', on='article')

In [55]:
def discount_index(discount):
    try:
        return 1 - discount / 100
    except:
        return 1

In [56]:
ecom.discount_value_promotions_1 = ecom.discount_value_promotions_1.apply(discount_index)

In [57]:
ecom['Цена со скидками Ecom'] = np.where(
    (ecom['price'] == ecom['discount_price']) & (ecom['discount_value_promotions_1'] == 1), ecom['discount_price'] * ecom['discount_value_promotions_1'], 
    np.where(
        ecom['discount_value_promotions_1'] == 0.5, ecom['discount_price'] * ecom['discount_value_promotions_1'], ecom['discount_price'] * ecom['discount_value_promotions_1'] * 0.8
    )
)

In [58]:
ecom['article'] = ecom['article'].astype(str)
ecom = ecom[['article', 'size', 'Цена со скидками Ecom']]

In [59]:
ecom = ecom.query('article in @tovs')

In [61]:
ecom.columns = ['ID Сайта', 'size', 'Цена со скидками Ecom']

In [62]:
result = result[result['ID Сайта'] != '']

In [63]:
result['ID Сайта'] = result['ID Сайта'].astype(int).astype(str)
result['Размер изделия'] = result['Размер изделия'].astype(str)

In [64]:
ecom['size'] = ecom['size'].fillna(0.0)
ecom['size'] = ecom['size'].astype(str)

In [65]:
ecom['merge'] = ecom['ID Сайта'] + ecom['size']

In [66]:
result['merge'] = result['ID Сайта'] + result['Размер изделия']

In [67]:
ecom = ecom.drop(columns=['ID Сайта', 'size'])

In [68]:
result = result.merge(ecom, how='outer', on='merge')

In [69]:
result = result.drop(columns=['merge'])

In [70]:
result = result.dropna(subset=['Код товара'])

In [71]:
result.columns =['Код склада', 'Код товара', 'Артикул поставщика', 'Внутренний артикул',
    'ID Сайта', 'Товарное направление', 'Товарная группа',
    'Товарный кластер', 'Проба', 'Размер изделия', 'Тип изделия 1',
    'Тип изделия 2', 'Тип изделия 3', 'Дизайн', 'Бренд',
    'Гендерный признак', 'Ценовая корзина', 'Группа цен', 'Чистый вес',
    'Цена изделия на бирке', 'Скидка на изделии', 'Запрет скидки',
    'Цена Закупки за 1 шт. (упр сс с НДС за единицу)', 'Дата закупки',
    'Опубликован', 'Ссылка на изображение', 'Остаток',
    'Количество продаж 14.03-11.06', 'Сумма продаж 14.03-11.06',
    'Чистый вес мин', 'Чистый вес медиана', 'Чистый вес макс',
    'Чистый вес размах, %', 'Цена изделия на бирке мин',
    'Цена изделия на бирке медиана', 'Цена изделия на бирке макс',
    'Базовая скидка мин', 'Базовая скидка макс', 'Цена со скидками мин',
    'Цена со скидками медиана', 'Цена со скидками макс',
    'Цена со скидками размах, %', 'Себестоимость мин',
    'Себестоимость медиана', 'Себестоимость макс',
    'Себестоимость размах, %', 'Наценка мин', 'Наценка медиана',
    'Наценка макс', 'Запрет к размещению', 'Цена со скидками Ecom']

In [72]:
df['Сумма в себестоимости на остатках'] = df['Остаток'] * df['Цена Закупки за 1 шт. (упр сс с НДС за единицу)']

In [73]:
summa_ost = df.groupby(by ='Код товара', as_index=False)['Сумма в себестоимости на остатках'].sum()

In [74]:
result = result.merge(summa_ost, how='left', on='Код товара')

In [75]:
df['Цена за грамм'] = df['Цена со скидками'] / df['Чистый вес']

In [76]:
price_per_gramm = df.groupby(by = 'Код товара', as_index=False)['Цена за грамм'].agg({'min', 'median', 'max'})
price_per_gramm.columns = ['Код товара', 'Цена за грамм мин', 'Цена за грамм медиана', 'Цена за грамм макс']

In [77]:
result = result.merge(price_per_gramm, how='left', on='Код товара')

In [78]:
result.to_csv('result.csv', index=False)

In [79]:
zapret_sk = df.groupby(by = 'Код товара', as_index=False)['Запрет скидки'].count()

In [81]:
yes = df[df['Запрет скидки'] == 'Да']
no = df[df['Запрет скидки'] == 'Нет']

In [82]:
yes = yes.groupby(by = 'Код товара', as_index=False)['Запрет скидки'].count()
no = no.groupby(by = 'Код товара', as_index=False)['Запрет скидки'].count()

In [83]:
yes.columns = ['Код товара', 'Количество с запрещенной скидкой']

In [84]:
no.columns = ['Код товара', 'Количество с разрешенной скидкой']

In [85]:
yes = yes.merge(no, how='outer', on='Код товара')

In [87]:
result = result.merge(yes, how='left', on='Код товара')

In [88]:
result = result.drop_duplicates(subset=['Код товара'])

In [93]:
df['Дата закупки'] = pd.to_datetime(df['Дата закупки'], dayfirst=True, format='mixed')

In [155]:
df['Старая закупка (до 31.12.2022)'] = np.where(
    df['Дата закупки'] <= pd.to_datetime('2022-12-31', dayfirst=True, format='mixed'), 1, 0
)

In [156]:
df['Новая закупка (с 01.01.2023)'] = np.where(
    df['Дата закупки'] >= pd.to_datetime('2023-01-01', dayfirst=True, format='mixed'), 1, 0
)

In [157]:
data = df.groupby(by = 'Код товара', as_index=False)[['Старая закупка (до 31.12.2022)', 'Новая закупка (с 01.01.2023)']].sum()

In [154]:
# df = df.drop(columns=['Старая закупка (до 31.12.2022)', 'Новая закупка (c 01.01.2023)',
#     'Новая закупка (с 01.01.2023)'])

In [160]:
data['Дата закупки'] = np.where(
    (data['Новая закупка (с 01.01.2023)'] > 0) & (data['Старая закупка (до 31.12.2022)'] > 0), 'Старые и новые',
    np.where(
        (data['Новая закупка (с 01.01.2023)'] > 0) & (data['Старая закупка (до 31.12.2022)'] == 0), 'Новые', 
        np.where(
            (data['Новая закупка (с 01.01.2023)'] == 0) & (data['Старая закупка (до 31.12.2022)'] > 0), 'Старые', 'Нет информации о дате'
        )
    )
)

In [164]:
data = data.drop(columns=['Старая закупка (до 31.12.2022)', 'Новая закупка (с 01.01.2023)'])

In [166]:
result = result.merge(data, how='left', on='Код товара')

In [170]:
result = result.drop(columns=['Дата закупки_x'])

In [171]:
result.columns = ['Код склада', 'Код товара', 'Артикул поставщика', 'Внутренний артикул',
    'ID Сайта', 'Товарное направление', 'Товарная группа',
    'Товарный кластер', 'Проба', 'Размер изделия', 'Тип изделия 1',
    'Тип изделия 2', 'Тип изделия 3', 'Дизайн', 'Бренд',
    'Гендерный признак', 'Ценовая корзина', 'Группа цен', 'Чистый вес',
    'Цена изделия на бирке', 'Скидка на изделии', 'Запрет скидки',
    'Цена Закупки за 1 шт. (упр сс с НДС за единицу)',
    'Опубликован', 'Ссылка на изображение', 'Остаток',
    'Количество продаж 14.03-11.06', 'Сумма продаж 14.03-11.06',
    'Чистый вес мин', 'Чистый вес медиана', 'Чистый вес макс',
    'Чистый вес размах, %', 'Цена изделия на бирке мин',
    'Цена изделия на бирке медиана', 'Цена изделия на бирке макс',
    'Базовая скидка мин', 'Базовая скидка макс', 'Цена со скидками мин',
    'Цена со скидками медиана', 'Цена со скидками макс',
    'Цена со скидками размах, %', 'Себестоимость мин',
    'Себестоимость медиана', 'Себестоимость макс',
    'Себестоимость размах, %', 'Наценка мин', 'Наценка медиана',
    'Наценка макс', 'Запрет к размещению', 'Цена со скидками Ecom',
    'Сумма в себестоимости на остатках', 'Цена за грамм мин',
    'Цена за грамм медиана', 'Цена за грамм макс',
    'Количество с запрещенной скидкой', 'Количество с разрешенной скидкой',
    'Дата закупки']

In [184]:
result['Чистый вес размах, %'] = result['Чистый вес размах, %'] * -1

In [197]:
result['Размах по весу'] = np.where(
    result['Чистый вес размах, %'] >= 0.3, '3. Большой размах (>30%)',
    np.where(
        result['Чистый вес размах, %'] >= 0.1, '2. Средний размах (10-30%)', '1. Небольшой размах (<10%)'
    )
)

In [196]:
result['Себестоимость размах, %'] = result['Себестоимость размах, %'] * -1

In [199]:
result['Размах по себестоимости'] = np.where(
    result['Себестоимость размах, %'] >= 0.5, '3. Большой размах (>50%)',
    np.where(
        result['Себестоимость размах, %'] >= 0.2, '2. Средний размах (20-50%)', '1. Небольшой размах (<10%)'
    )
)

In [201]:
result['Цена со скидками размах, %'] = result['Цена со скидками размах, %'] * -1

In [203]:
result['Размах по цене со скидками'] = np.where(
    result['Цена со скидками размах, %'] >= 0.5, '3. Большой размах (>50%)',
    np.where(
        result['Цена со скидками размах, %'] >= 0.2, '2. Средний размах (20-50%)', '1. Небольшой размах (<10%)'
    )
)

In [207]:
result['Цена за грамм размах, %'] = (result['Цена за грамм макс'] - result['Цена за грамм мин']) / ((result['Цена за грамм макс'] + result['Цена за грамм мин']) / 2)

In [208]:
result['Цена за грамм размах, %'] = result['Цена за грамм размах, %'] * -1

In [210]:
result['Размах по цене за грамм'] = np.where(
    result['Цена за грамм размах, %'] >= 0.3, '3. Большой размах (>30%)',
    np.where(
        result['Цена за грамм размах, %'] >= 0.1, '2. Средний размах (10-30%)', '1. Небольшой размах (<10%)'
    )
)

In [258]:
result['Скидка разрешена'] = result['Количество с запрещенной скидкой'] + result['Количество с разрешенной скидкой']

In [267]:
result['Скидка разрешена'] = np.where(
    result['Скидка разрешена'].astype(str).str.contains('nan'), 'Да или нет', 'Да и нет'
)

In [272]:
result.columns

Index(['Код склада', 'Код товара', 'Артикул поставщика', 'Внутренний артикул',
       'ID Сайта', 'Товарное направление', 'Товарная группа',
       'Товарный кластер', 'Проба', 'Размер изделия', 'Тип изделия 1',
       'Тип изделия 2', 'Тип изделия 3', 'Дизайн', 'Бренд',
       'Гендерный признак', 'Ценовая корзина', 'Группа цен', 'Чистый вес',
       'Цена изделия на бирке', 'Скидка на изделии', 'Запрет скидки',
       'Цена Закупки за 1 шт. (упр сс с НДС за единицу)', 'Опубликован',
       'Ссылка на изображение', 'Остаток', 'Количество продаж 14.03-11.06',
       'Сумма продаж 14.03-11.06', 'Чистый вес мин', 'Чистый вес медиана',
       'Чистый вес макс', 'Чистый вес размах, %', 'Цена изделия на бирке мин',
       'Цена изделия на бирке медиана', 'Цена изделия на бирке макс',
       'Базовая скидка мин', 'Базовая скидка макс', 'Цена со скидками мин',
       'Цена со скидками медиана', 'Цена со скидками макс',
       'Цена со скидками размах, %', 'Себестоимость мин',
       'Себестои

In [274]:
result['Цена изделия на бирке размах, %'] = (result['Цена изделия на бирке макс'] - result['Цена изделия на бирке мин']) / ((result['Цена изделия на бирке макс'] + result['Цена изделия на бирке мин']) / 2)

In [276]:
result['Цена изделия на бирке размах, %'] = result['Цена изделия на бирке размах, %'] * -1

In [279]:
result['Размах цена изделия на бирке'] = np.where(
    result['Цена изделия на бирке размах, %'] >= 0.5, '3. Большой размах (>50%)',
    np.where(
        result['Цена изделия на бирке размах, %'] >= 0.2, '2. Средний размах (20-50%)', '1. Небольшой размах (<20%)'
    )
)

In [281]:
result.to_excel(r'C:\Users\Trenkin.Sergey\Desktop\notebooks\result.xlsx', index=False)

In [265]:
# result[result['Товарное направление'].str.contains('ПДК|ДК|ИФ')].to_excel(r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\Исследование цен\Исследование цен ПДК ДК ИФ 18.06.24.xlsx', index=False)

In [266]:
# result[result['Товарное направление'].str.contains('БК|ЦБ')].to_excel(r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\Исследование цен\Исследование цен БК ЦБ 18.06.24.xlsx', index=False)